## Data cleaning

In [87]:
import numpy as np
import pandas as pd
%matplotlib inline 

import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm as anova

from sklearn.linear_model import LinearRegression;

# from EDA_plots import EDA_plots

# housing = pd.read_csv("C:/Users/kiwym/operation_goldfish/data", index_col=0)
# housing = pd.read_csv("C:/Users/kiwym/operation_goldfish/data/Ames_Housing_Price_Data_raw.csv", index_col=0);
housing = pd.read_csv("../data/person2data.csv", index_col=0);
# mask = housing["BsmtFullBath"].isnull();
# housing = housing.fillna(0);


In [88]:
df = data_cleaning_part_2(housing);
df.shape

C:\Users\kiwym\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(2580, 42)

In [84]:
# PIDs 903230120 and 908154080 had no basement room info.
# Let's just assume that they do not have basement at all.

df = housing.drop("PID", axis = 1);
# df = housing[["SalePrice", "MSSubClass", "BldgType", "HouseStyle", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",\
#              "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd",\
#              "Functional", "SaleCondition", "SaleType", "MoSold", "YrSold"]];
df = df.reset_index(drop= True)
df.shape

(2580, 20)

## Brief classification of the columns in my part:

#### Nominal:
- MSSubClass: Type of dwelling
- HouseStyle: Style of dwelling
- SaleCondition: Condition of sale (like how is it sold)
- SaleType: Type of sale

#### Ordinal:
Functional: Home functionality (Assume typical unless deductions are warranted) <- Typ (Good) ... Sal (Bad)


#### Continuous:
- 1stFlrSF: First Floor square feet
- 2ndFlrSF: Second floor square feet <- Should ignore zeros when regression
- LowQualFinSF: Low quality finished square feet (all floors) <- Should ignore zeros when regression
- GrLivArea: Above grade (ground) living area square feet <- Might be extraneous, = 1stFlrSF + 2ndFlrSF

#### Discrete:
- BsmtFullBath: Basement full bathrooms
- BsmtHalfBath: Basement half bathrooms
- FullBath: Full bathrooms above grade
- HalfBath: Half baths above grade
- BedroomAbvGr: Bedrooms above grade (does NOT include basement bedrooms)
- KitchenAbvGr: Kitchens above grade
- TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

- MoSold: Month Sold (MM)

- YrSold: Year Sold (YYYY)


In [76]:
# Ordinal variable handling
functionality_dict={
    "Typ": 7, # Typical Functionality
    "Min1": 6, # Minor Deductions 1
    "Min2": 5, # Minor Deductions 2
    "Mod": 4, # Moderate Deductions
    "Maj1": 3, # Major Deductions 1
    "Maj2": 2, # Major Deductions 2
    "Sev": 1, # Severely Damaged
    "Sal": 0, # Salvage only
};
df.loc[df["Functional"].isna(), "Functional"]='0';
df["Functional_dis"]=df["Functional"].map(lambda x: functionality_dict[x]);
df = df.rename(columns = {"Functional": "Functional_ord"});

In [83]:
# Create a dictionary which describes the data type of each columns.
type_dict_person_2={
    "MSSubClass": "Nominal",
    "HouseStyle": "Nominal",
    "SaleCondition": "Nominal",
    "SaleType": "Nominal",
    "Functional_ord": "Ordinal", # Changed from "Functional"
    "1stFlrSF": "Continuous",
    "2ndFlrSF": "Continuous",
    "LowQualFinSF": "Continuous", # Rejectable p-value
    "GrLivArea": "Continuous",
    "BsmtFullBath": "Discrete",
    "BsmtHalfBath": "Discrete", # Rejectable p-value
    "FullBath": "Discrete",
    "HalfBath": "Discrete",
    "BedroomAbvGr": "Discrete",
    "KitchenAbvGr": "Discrete",
    "TotRmsAbvGrd": "Discrete",
    "MoSold": "Discrete", # Rejectable p-value
    "YrSold": "Discrete", # Rejectable p-value
    ####### Below are columns created by myself #######
    "Functional_dis": "Discrete", # Functional in a (Salvage) 0-7 (Full) scale.
    "1stFlrSF_log": "Continuous",
    "2ndFlrSF_log": "Continuous",
    "GrLivArea_log": "Continuous",
    "BsmtEqBath_0.3": "Continuous", # Equivalent number of bathrooms in basement, treating half as 0.3
    "EqBath_0.3": "Continuous", # Equivalent number of bathrooms in basement, treating half as 0.5
    "BsmtEqBath_0.5": "Continuous", # Equivalent number of bathrooms over ground, treating half as 0.3
    "EqBath_0.5": "Continuous", # Equivalent number of bathrooms over ground, treating half as 0.5
    "1-Story": "Nominal",
    "1.5-Story": "Nominal",
    "2-Story": "Nominal",
    "2.5-Story": "Nominal",
    "SplitMulti": "Nominal",
    "2FamConv": "Nominal",
    "SptFoyer": "Nominal",
    "Duplex": "Nominal", # These 7 attributes are exclusive
    "Unfinished": "Nominal",
    "PUD": "Nominal",
    "1Fam": "Nominal",
    "TwnhsE": "Nominal",
    "TwnhsI": "Nominal" # These 5 tags can coexist with the 7 attributes in the top.
};

## The all-in-one pack

In [89]:
def data_cleaning_part_2(housing):
    '''
    Input variable:
    housing: a compatible dataframe.
    
    Description:
    Assume that housing is the dataframe directly imported from person2.csv,
    the function fills up the NA values and add some columns Hao-Wei felt necessary.
    For an explanation of the columns, see the dictionary above and the original description file.
    '''
    df = housing.fillna(0);
    # df = df.drop("PID", axis = 1);
    df = df.reset_index(drop= True);
    
    # Ordinal variable handling
    functionality_dict={
        "Typ": 7, # Typical Functionality
        "Min1": 6, # Minor Deductions 1
        "Min2": 5, # Minor Deductions 2
        "Mod": 4, # Moderate Deductions
        "Maj1": 3, # Major Deductions 1
        "Maj2": 2, # Major Deductions 2
        "Sev": 1, # Severely Damaged
        "Sal": 0, # Salvage only
    };
    df.loc[df["Functional"].isna(), "Functional"]='0';
    df["Functional_dis"]=df["Functional"].map(lambda x: functionality_dict[x]);
    df = df.rename(columns = {"Functional": "Functional_ord"}); # 21
    
    # Adding columns with log scales

    temp = pd.DataFrame({"1stFlrSF_log": np.log10(df["1stFlrSF"]),
                        "2ndFlrSF_log": np.log10(df["2ndFlrSF"]),
                        "GrLivArea_log": np.log10(df["GrLivArea"])});
    df = pd.concat([df, temp], axis = 1); # 24
    
    # Add weight columns for bathrooms
    half_equiv = [0.3, 0.5];

    temp_dict = {};
    for eq in half_equiv:
        temp_dict["BsmtEqBath_"+"{:.1f}".format(eq)] = df["BsmtFullBath"] + eq*df["BsmtHalfBath"];
        temp_dict["EqBath_"+"{:.1f}".format(eq)] =  df["FullBath"] + eq*df["HalfBath"];

    temp = pd.DataFrame(temp_dict);
    df = pd.concat([df, temp], axis = 1); # 28

    # Extract nominal columns for better interpretation.
    temp_dict = {};
    temp_dict["1-Story"]    = df.apply(lambda x: x["MSSubClass"] in [20, 30, 40, 120], axis=1);
    temp_dict["1.5-Story"]  = df.apply(lambda x: x["MSSubClass"] in [45, 50, 150], axis=1);
    temp_dict["2-Story"]    = df.apply(lambda x: x["MSSubClass"] in [60, 70, 160], axis=1);
    temp_dict["2.5-Story"]  = df.apply(lambda x: x["MSSubClass"] == 75, axis=1);
    temp_dict["SplitMulti"] = df.apply(lambda x: x["MSSubClass"] in [80, 180], axis=1);
    temp_dict["2FamConv"]   = df.apply(lambda x: x["MSSubClass"] == 190, axis=1);
    temp_dict["SptFoyer"]   = df.apply(lambda x: x["MSSubClass"] == 85, axis=1);
    temp_dict["Duplex"]     = df.apply(lambda x: x["MSSubClass"] == 90, axis=1);
    temp_dict["Unfinished"] = df.apply(lambda x: x["MSSubClass"] == 190, axis=1) | df.apply(lambda x: x["HouseStyle"] in ["1.5Unf", "2.5Unf"], axis=1);
    temp_dict["PUD"]        = df.apply(lambda x: x["MSSubClass"] in [120, 150, 160, 180], axis=1);
    temp_dict["1Fam"]       = df.apply(lambda x: x["BldgType"] == "1Fam", axis=1);
    temp_dict["TwnhsE"]     = df.apply(lambda x: x["BldgType"] == "TwnhsE", axis=1);
    temp_dict["TwnhsI"]     = df.apply(lambda x: x["BldgType"] == "TwnhsI", axis=1);

    temp = pd.DataFrame(temp_dict).astype(int);
    df = pd.concat([df, temp], axis = 1); # 41, 42 if PID not dropped
    
    return df;

## House class and sell price

Here are some ideas to generate meaningful columns

From the column `MSSubClass`
- Stories: 1/2/1.5/2.5/Spli
- Unfinished?
- Split foyer/multileveled?
- PUD?

## House area and sale price

In [78]:
# Adding columns with log scales

temp = pd.DataFrame({"1stFlrSF_log": np.log10(df["1stFlrSF"]),
                    "2ndFlrSF_log": np.log10(df["2ndFlrSF"]),
                    "GrLivArea_log": np.log10(df["GrLivArea"])});
df = pd.concat([df, temp], axis = 1)

C:\Users\kiwym\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Room issues and sale price

In [79]:
# Add weight columns for bathrooms
half_equiv = [0.3, 0.5];

temp_dict = {};
for eq in half_equiv:
    temp_dict["BsmtEqBath_"+"{:.1f}".format(eq)] = df["BsmtFullBath"] + eq*df["BsmtHalfBath"];
    temp_dict["EqBath_"+"{:.1f}".format(eq)] =  df["FullBath"] + eq*df["HalfBath"];

temp = pd.DataFrame(temp_dict);
df = pd.concat([df, temp], axis = 1);



In [80]:
# Extract nominal columns for better interpretation.
temp_dict = {};
temp_dict["1-Story"]    = df.apply(lambda x: x["MSSubClass"] in [20, 30, 40, 120], axis=1);
temp_dict["1.5-Story"]  = df.apply(lambda x: x["MSSubClass"] in [45, 50, 150], axis=1);
temp_dict["2-Story"]    = df.apply(lambda x: x["MSSubClass"] in [60, 70, 160], axis=1);
temp_dict["2.5-Story"]  = df.apply(lambda x: x["MSSubClass"] == 75, axis=1);
temp_dict["SplitMulti"] = df.apply(lambda x: x["MSSubClass"] in [80, 180], axis=1);
temp_dict["2FamConv"]   = df.apply(lambda x: x["MSSubClass"] == 190, axis=1);
temp_dict["SptFoyer"]   = df.apply(lambda x: x["MSSubClass"] == 85, axis=1);
temp_dict["Duplex"]     = df.apply(lambda x: x["MSSubClass"] == 90, axis=1);
temp_dict["Unfinished"] = df.apply(lambda x: x["MSSubClass"] == 190, axis=1) | df.apply(lambda x: x["HouseStyle"] in ["1.5Unf", "2.5Unf"], axis=1);
temp_dict["PUD"]        = df.apply(lambda x: x["MSSubClass"] in [120, 150, 160, 180], axis=1);
temp_dict["1Fam"]       = df.apply(lambda x: x["BldgType"] == "1Fam", axis=1);
temp_dict["TwnhsE"]     = df.apply(lambda x: x["BldgType"] == "TwnhsE", axis=1);
temp_dict["TwnhsI"]     = df.apply(lambda x: x["BldgType"] == "TwnhsI", axis=1);

temp = pd.DataFrame(temp_dict).astype(int);
df = pd.concat([df, temp], axis = 1);

In [81]:
df.head()

,SalePrice,MSSubClass,BldgType,HouseStyle,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,...,2.5-Story,SplitMulti,2FamConv,SptFoyer,Duplex,Unfinished,PUD,1Fam,TwnhsE,TwnhsI
0,126000,30,1Fam,1Story,856,0,0,856,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,139500,120,TwnhsE,1Story,1049,0,0,1049,1.0,0.0,...,0,0,0,0,0,0,1,0,1,0
2,124900,30,1Fam,1Story,1001,0,0,1001,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,114000,70,1Fam,2Story,717,322,0,1039,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,227000,60,1Fam,2Story,810,855,0,1665,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
